In [ ]:
"""
Further suppose that the dataset is ticker-tagged for 200 equities which respond to the dataset in roughly similar ways. Maybe the economic basis of the data is shared by all the equities, or maybe investor trading behavior on the data unites them.

"""

import numpy as np
import pandas as pd

# simulate the returns and signal
r = 0 # avg return
r_sig = 1 # avg return conditioned on signal
sigma_alpha = 0.5 # alpha std dev
sigma_beta = 0.3 # beta std dev
rho = 0.4 # correlation between alpha and beta

# Means
Mu = np.array([r, r_sig])

# covariance
Cov = np.diag([r, r_sig]) @ np.array([1,rho,rho,1]).reshape(2,2) @ np.diag([r, r_sig])

print(Mu)
print(Cov)

In [ ]:
"""
More formally we can say the alt dataset comprises an explanatory factor portfolio, and the equity returns have betas to that factor. That implies a covariance structure between the equities we can exploit for superior signal construction. So we continue simulating this data...
"""

# generate population equity data
m_symbols = 200

np.random.seed(60)
mvr = np.random.multivariate_normal(Mu, Cov, m_symbols)

alpha_ret = mvr[:,0]
beta_ret = mvr[:,1]

print(mvr)

In [ ]:
# simulate 50 return samples for each symbol - train set

np.random.seed(64)
n_samples = 50

event = np.random.uniform(0,1, int(n_samples * m_symbols))
sym_id = np.repeat(list(range(1, m_symbols + 1)), n_samples)
mu = alpha_ret[sym_id - 1] + beta_ret[sym_id - 1] * event
sigma = 0.5

ret = np.random.normal(mu, sigma, m_symbols * n_samples)
train_df = pd.DataFrame({'symbol': sym_id, 'event':event, 'ret': ret})
train_df

In [ ]:
"""
Now we have a train set and a test set. First let's try fitting an OLS model to each equity based on only its sample of the data, individually. We'll estimate the model on the train set, then evaluate it via MSE on the test set (MSE chosen over MAE to penalize very high errors).
"""

# simulate 20 return samples for each symbol - test set

np.random.seed(32)
n_samples = 20

event = np.random.uniform(0,1, int(n_samples * m_symbols))
sym_id = np.repeat(list(range(1, m_symbols + 1)), n_samples)
mu = alpha_ret[sym_id - 1] + beta_ret[sym_id - 1] * event
sigma = 0.5

ret = np.random.normal(mu, sigma, m_symbols * n_samples)
test_df = pd.DataFrame({'symbol': sym_id, 'event':event, 'ret': ret})
test_df

In [ ]:
"""
We evaluate the per-symbol models ("no pooling") and average their MSEs, then evaluate out of sample on the entire dataset ("complete pooling"). Per-symbol the average MSE is ~0.604, total is 1.21. Can we do better than this? Let's try estimating the model with complete pooling.
"""

import statsmodels.api as sm

symbol_mse = []
total_y = []
total_y_hat = []
rsq = []
pval = []

for symbol in train_df['symbol'].drop_duplicates().to_list():
    train_data = train_df.loc[train_df['symbol'] == symbol][['event', 'ret']].to_numpy()
    y = train_data.T[1].reshape(-1,1)
    X = sm.add_constant(train_data.T[0])
    model = sm.OLS(y, X).fit()
    rsq.append(model.rsquared)
    pval.append(model.pvalues)

    # evaluate the model via MSE
    test_data = test_df.loc[test_df['symbol'] == symbol][['event', 'ret']].to_numpy()
    y = test_data.T[1].reshape(-1,1)
    X = sm.add_constant(test_data.T[0])

    y_hat = model.predict(exog=X)

    mse = np.square(y - y_hat).mean()
    total_y += [k for k in y]
    total_y_hat += [k for k in y_hat]
    symbol_mse.append(mse)

# measure avg of MSE across per-symbol models
np.mean(symbol_mse)


In [ ]:
# also measure MSE across entire dataset - no pooling performs much worse on entire dataset than per-symbol

np.square(np.array(total_y) - np.array(total_y_hat)).mean()

In [ ]:
"""
The model estimates the alt dataset's signal across the entire sample of equities without any conditioning on equities. MSEs on a per-symbol basis are worse than before, but the total dataset MSE has improved. Can we do better than this?
"""

pooled_mse = []

train_data = train_df[['event', 'ret']].to_numpy()
y = train_data.T[1].reshape(-1,1)
X = sm.add_constant(train_data.T[0])
model = sm.OLS(y, X).fit()
print(model.summary())

In [ ]:
for symbol in train_df['symbol'].drop_duplicates().to_list():
    # evaluate the model via MSE

    test_data = test_df.loc[test_df['symbol'] == symbol][['event', 'ret']].to_numpy()
    y = test_data.T[1].reshape(-1,1)
    X = sm.add_constant(test_data.T[0])

    #y_hat = X @ model.params.T
    y_hat = model.predict(exog=X)

    mse = np.square(y - y_hat).mean()
    pooled_mse.append(mse)

np.mean(pooled_mse)

In [ ]:
# also measure MSE across entire dataset - total pooling performs much better on entire dataset than per-symbol

X = sm.add_constant(test_df['event'].to_numpy())
y_hat = model.predict(X)
np.square(y-y_hat).mean()

In [ ]:
"""
In fact, we can fit a multilevel regression model instead. The "fixed effects" have an interpretation as the completely pooled regression coefficient, and the "random effects" are per-symbol deviations. To make a per-symbol prediction, we sum the effects. The MSEs improve.
"""

import statsmodels.formula.api as smf

# build mixed effects model - partial pooling on symbols

model = smf.mixedlm('ret ~ event', train_df, groups=train_df['symbol'])
md = model.fit()
re = {symbol: md.random_effects[symbol][0] for symbol in md.random_effects}
test_df['fe'] = md.predict(exog=test_df)
test_df['re'] = test_df['symbol'].map(re)
test_df['ret_hat'] = test_df['fe'] + test_df['re']

# evaluate the model per-symbol - it outperforms both the unpooled and completely pooled versions

mse = []

for symbol in test_df['symbol'].drop_duplicates().to_list():
    test_data = test_df.loc[test_df['symbol'] == symbol]
    sym_mse = np.square(test_data['ret'] - test_data['ret_hat']).mean()
    mse.append(sym_mse)

np.mean(mse)

In [ ]:
# evaluate the MSE across the entire dataset - it outperforms both the unpooled and completely pooled versions

me_mse = np.square(test_df['ret'] - test_df['ret_hat']).mean()
me_mse

In [ ]:
"""
Aside from improved out-of-sample accuracy, this comes with an elegant basis for portfolio construction. The random effects have a natural interpretation as the relative out (under) performance of each symbol, neutral to the group/cluster on which the model was estimated.

So for each alt dataset with which this modeling approach works, you can simple sort by random effects in each group, long the top decile of random effects, short the bottom decile of random effects, and hedge the remainder to trade a neutral basket.
"""

import quantstats as qs

# Sort within each symbol group by random effects
train_df = train_df.sort_values(by=['event'])

# Define function to classify deciles
def classify_decile(group):
    group['decile'] = pd.qcut(group['event'], 10, labels=False)
    return group

# Apply the function to the DataFrame
train_df = train_df.groupby('symbol', group_keys=False).apply(classify_decile)

# Create long and short positions
train_df['position'] = 0
train_df.loc[train_df['decile'] == 9, 'position'] = 1  # Long top decile
train_df.loc[train_df['decile'] == 0, 'position'] = -1  # Short bottom decile

# Hedge the remainder to trade a neutral basket
train_df['hedged_position'] = train_df.groupby('symbol')['position'].transform(lambda x: x - x.mean())

# Calculate the strategy returns
train_df['strategy_ret'] = train_df['hedged_position'] * train_df['ret']

# Aggregate strategy returns
strategy_returns = train_df.groupby('symbol')['strategy_ret'].sum()

# Convert to cumulative returns for QuantStats analysis
strategy_returns

In [ ]:
strategy_returns.plot()